In [1]:
import cimgraph.data_profile.cimhub_2023 as cim
from uuid import UUID
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import BlazegraphConnection
from cimgraph.models import FeederModel
from cimgraph.models import DistributedArea
import cimgraph.queries.sparql as sparql
import cimgraph.utils as utils
from defusedxml.ElementTree import parse
cim_profile = 'cimhub_2023'

In [2]:
from cimgraph.databases.fileparsers import XMLFile
params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)
xml_file = XMLFile(params)
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=xml_file, container=feeder, distributed=False)

In [15]:
utils.write_xml(network, 'out13.xml')

In [16]:
from cimgraph.databases.fileparsers import XMLFile
params2 = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)
xml_file2 = XMLFile(params2)
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder2 = cim.Feeder(mRID = feeder_mrid)
network2 = FeederModel(connection=xml_file2, container=feeder2, distributed=False)

In [22]:
for cim_class in network.graph:
    if cim_class not in network2.graph:
        print(f'{cim_class.__name__} not in new')
    else:
        for uuid in network.graph[cim_class]:
            if uuid not in network2.graph[cim_class]:
                print(f'{cim_class.__name__} : {str(uuid)} not in new')
            else:
                for attribute in cim_class.__dataclass_fields__:
                    value1 = str(getattr(network.graph[cim_class][uuid], attribute))
                    value2 = str(getattr(network2.graph[cim_class][uuid], attribute))
                    if value1 != value2:
                        print(f'{cim_class.__name__} : {str(uuid)}')
                        print(value1)
                        print(value2)
                        print()


Location : 58246081-fedb-4de7-8173-c408cfe3803e
[{"@id": "5143b56c-5af7-4d47-a6b1-5f952b891190", "@type": "ACLineSegmentPhase"}, {"@id": "22182853-3560-4ee4-8ba2-0730c5f5af94", "@type": "ACLineSegmentPhase"}, {"@id": "d34b0d01-b082-4081-a3cc-b68b9b8313a4", "@type": "ACLineSegment"}]
[{"@id": "d34b0d01-b082-4081-a3cc-b68b9b8313a4", "@type": "ACLineSegment"}, {"@id": "5143b56c-5af7-4d47-a6b1-5f952b891190", "@type": "ACLineSegmentPhase"}, {"@id": "22182853-3560-4ee4-8ba2-0730c5f5af94", "@type": "ACLineSegmentPhase"}]

Location : fc973b3e-2ac2-4c61-804d-6cf40fff3cc0
[{"@id": "487611d2-1bbf-4b65-8fa0-ae3b8637e32c", "@type": "EnergyConsumerPhase"}, {"@id": "3b2021a7-4bfc-418d-9c20-bd6838e52cf8", "@type": "EnergyConsumer"}]
[{"@id": "3b2021a7-4bfc-418d-9c20-bd6838e52cf8", "@type": "EnergyConsumer"}, {"@id": "487611d2-1bbf-4b65-8fa0-ae3b8637e32c", "@type": "EnergyConsumerPhase"}]

BaseVoltage : 2a158e0c-cd01-4a50-aeba-59d761fcf15d
[{"@id": "a9de8829-58cb-4750-b2a2-672846a89753", "@type": "Line

{'identifier': Field(name='identifier',type='Optional[UUID]',default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x7fdb5890ba90>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'type': 'Attribute', 'minOccurs': '1', 'maxOccurs': '1'}),kw_only=False,_field_type=_FIELD),
 'mRID': Field(name='mRID',type='Optional[str]',default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x7fdb5890ba90>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'type': 'Attribute', 'minOccurs': '0', 'maxOccurs': '1', 'docstring': '\n                Master resource identifier issued by a model authority. The mRID is unique\n                within an exchange context. Global uniqueness is easily achieved by using\n                a UUID, as specified in RFC 4122, for the mRID. The use of UUID is strongly\n                recommended.\n                For CIMXML data files in RDF syntax conforming to IEC 61970-552 Edition\n                1, the mRID is map

In [8]:
graph = {}
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=7)
database = BlazegraphConnection(params)

In [9]:
rdf = '''{http://www.w3.org/1999/02/22-rdf-syntax-ns#}'''
# Parse the XML file
# tree = parse('./test_models/ieee13_assets.xml')
tree = parse('/home/ande188/CIM-Graph/tests/test_models/ieee9500bal.xml')
root = tree.getroot()

class_index = {}

# # Function to extract tag without namespace
# def extract_tag(tag):
#     return tag.split('}')[-1]

# Iterate over the elements and create dataclass instances
for element in root:

    class_name = element.tag.split('{'+params.namespace+'}')[1]
    cim_class = eval(f'cim.{class_name}')

    if class_name in cim.__all__:
        uri = element.get(f'{rdf}about')
        identifier = UUID(uri.strip('_').lower())
        uri = uri.split(':')[-1]  # Extract UUID from the full URI
        class_index[identifier] = cim_class
        obj = database.create_object(graph, cim_class, uri)

# for element in root:      
#     class_name = element.tag.split('{'+params.namespace+'}')[1]
#     cim_class = eval(f'cim.{class_name}')

#     if class_name in cim.__all__:
#         uri = element.get(f'{rdf}about')
        
#         identifier = UUID(uri.strip('_').lower())
#         uri = uri.split(':')[-1]  # Extract UUID from the full URI
        
#         obj = graph[cim_class][UUID(uri)]
#         for sub_element in element:
#             sub_tag = sub_element.tag.split('}')[-1]
#             association = database.check_attribute(cim_class, sub_tag)
#             try:
#                 edge_uri = sub_element.attrib[f'{rdf}resource'].split('uuid:')[-1]
#             except:
#                 edge_uri = None

#             if edge_uri is not None:
#                 if database.namespace not in edge_uri:
#                     edge_uuid = UUID(edge_uri.strip('_').lower())                  
#                     edge_class = class_index[edge_uuid]
#                     database.create_edge(graph, cim_class, identifier, sub_tag, edge_class, edge_uri)
#                     reverse = cim_class.__dataclass_fields__[association].metadata['inverse']
#                     database.create_edge(graph, edge_class, edge_uuid, reverse, cim_class, graph[cim_class][identifier].uri())
#                 else:
#                     enum_text = edge_uri.split(database.namespace)[1]
#                     enum_text = enum_text.split('>')[0]
#                     enum_class = enum_text.split('.')[0]
#                     enum_value = enum_text.split('.')[1]
#                     edge_enum = eval(f'cim.{enum_class}(enum_value)')
#                     if association is not None:
#                         setattr(graph[cim_class][identifier], association, edge_enum)
#             else:
#                 if association is not None:
#                     setattr(obj, association, sub_element.text)


In [10]:
len(graph[cim.ACLineSegment].keys())

3912

In [11]:
import multiprocessing
import defusedxml.ElementTree as ET

# Parse the XML file
def parse_xml_file(file_path):
    tree = ET.parse(file_path)
    return tree.getroot()

# Extract tag without namespace
def extract_tag(tag):
    return tag.split('}')[-1]

# Function to create the object and set attributes
def process_element(element, element_to_class, graph, lock):
    tag = extract_tag(element.tag)
    data_class = element_to_class.get(tag)
    if data_class:
        uuid = element.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about')
        uuid = uuid.split(':')[-1]  # Extract UUID from the full URI
        with lock:
            if uuid not in graph:
                graph[uuid] = xml_file.create_object(graph, data_class, uuid) #data_class(mRID=uuid)
        obj = graph[uuid]

        data_class in cim.__all__
        
        for sub_element in element:
            sub_tag = extract_tag(sub_element.tag)
            if sub_tag.startswith('IdentifiedObject.'):
                attribute = sub_tag.split('.')[-1]
                with lock:
                    setattr(obj, attribute, sub_element.text)
                    
            elif sub_tag.startswith(tag + '.'):
                attribute = sub_tag.split('.')[-1]
                with lock:
                    setattr(obj, attribute, sub_element.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource'))


In [12]:
def parse_nodes(graph, element, lock):
        # Iterate over the elements and create dataclass instances
        # for element in self.root:
        class_name = element.tag.split('{'+self.namespace+'}')[1]

        if class_name in self.cim.__all__:
            print(class_name)
            cim_class = eval(f'self.cim.{class_name}')
            uri = element.get(f'{self.rdf}about')
            identifier = UUID(uri.strip('_').lower())
            uri = uri.split(':')[-1]  # Extract UUID from the full URI
            self.class_index[identifier] = cim_class
            with lock:
                obj = self.create_object(graph, cim_class, uri)
        else:
            # _log.warning(f'{class_name} not in data profile')
            pass
        return None

In [13]:
def main():
    # Define the mapping from XML element names to dataclass types
    element_to_class = {
        'GeographicalRegion': cim.GeographicalRegion,
        'SubGeographicalRegion': cim.SubGeographicalRegion,
        'Substation': cim.Substation,
        'Location': cim.Location,
        'Feeder': cim.Feeder
    }

    xml_file_path = '/home/ande188/CIM-Graph/tests/test_models/ieee13.xml'
    root = parse_xml_file(xml_file_path)
    
    with multiprocessing.Manager() as manager:
        graph = manager.dict()
        lock = manager.Lock()

        # # Use ProcessPoolExecutor to process elements in parallel
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        #     pool.starmap(
        #         process_element, 
        #         [(element, element_to_class, graph, lock) for element in root]
        #     )
        with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
            pool.starmap(parse_nodes, [(graph, element, lock) for element in root])
            
        # Example of accessing created objects (for debugging or verification)
        for uuid, obj in graph.items():
            print(uuid, obj)

In [14]:
main()

NameError: name 'self' is not defined